Created on Mon Jan 6 09:58:13 2020
<br>
Group 7
<br>
@authors: M.D., C.D., E.G.

In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib.pyplot import imread

In [2]:
project_path = './../'
data_path = 'Interpromo2020/All Data/ANALYSE IMAGE/IMG SEATGURU/'
path = project_path + data_path
img_list = os.listdir(path)
nb_images = len(img_list)

## Dataframe

In [3]:
# List of all matrices (images)
imgs = list()
for i in range(nb_images):
    img = imread(path + img_list[i])
    imgs.append(img)

# Init the dataframe that will contain all basic info about images
imgs_df = pd.DataFrame(columns = ['name', 'format', 'height', 'width',
                                  'height_to_width', 'ncol', 'aircraft_manufacturer', 'aircraft_type'])
imgs_df.name = img_list

In [4]:
def get_imgs_formats(imgs: list, imgs_df: pd.DataFrame) -> pd.DataFrame:
    """Fills the DataFrame with pictures info (format and dimensions).
    
    Parameters:
    imgs: list of arrays representing images
    imgs_df: empty DataFrame with at least a 'name' column

    Out:
    imgs_df: DataFrame with 4 new columns: 'format', 'height', 'width',
             'height_to_width', 'ncol', and one line per image

    """

    # Formats
    imgs_df['format'] = imgs_df.name.apply(lambda x: x.split('.')[1])

    # Shapes
    heights = [imgs[k].shape[0] if len(imgs[k].shape) != 0 else 0 for k in range(len(imgs))]
    widths = [imgs[k].shape[1] if len(imgs[k].shape) != 0 else 0 for k in range(len(imgs))]
    ncols = [imgs[k].shape[2] if len(imgs[k].shape) != 0 else 0 for k in range(len(imgs))]

    imgs_df['height'] = heights
    imgs_df['width'] = widths
    imgs_df['height_to_width'] = [x / y if y != 0 else 0 for x, y in zip(imgs_df.height, imgs_df.width)]
    imgs_df['ncol'] = ncols

    return imgs_df

In [5]:
# Apply function to fill in DataFrame
imgs_df = get_imgs_formats(imgs, imgs_df)

# Aircraft manufacturers
aircraft_manufacturers = ['Airbus' if 'Airbus' in imgs_df.name[k] 
                          else 'Boeing' if 'Boeing' in imgs_df.name[k]
                          else 'Other' for k in range(len(imgs_df))]

# Aircraft types
aircraft_types = [name.split('_')[name.split('_').index(aircraft_manufacturer) + 1].split('-')[0]
                  if aircraft_manufacturer in name.split('_') else ''
                  for name, aircraft_manufacturer in zip(imgs_df.name, aircraft_manufacturers)]

# Add missing As for Airbus aircrafts
aircraft_types = [aircraft_types[k] 
                  if ('A' in aircraft_types[k] or '7' in aircraft_types[k] or aircraft_types[k] == '')
                  else 'A' + aircraft_types[k] for k in range(len(aircraft_types))]

# Remove Airbus 'neos' because we don't need that much detail
aircraft_types = [aircraft_types[k].replace('neo', '') for k in range(len(aircraft_types))]

# Remove Ds
aircraft_types = [aircraft_types[k].replace('D', '') for k in range(len(aircraft_types))]

# Fill in dataframe columns
imgs_df.aircraft_manufacturer = aircraft_manufacturers
imgs_df.aircraft_type = aircraft_types

In [6]:
imgs_df.head()

,name,format,height,width,height_to_width,ncol,aircraft_manufacturer,aircraft_type
0,Aegean_Airlines_Airbus_A320-200_0.jpg,jpg,720,720,1.000000,3,Airbus,A320
1,Aegean_Airlines_Airbus_A320-200_1.jpg,jpg,720,720,1.000000,3,Airbus,A320
2,Aegean_Airlines_Airbus_A320-200_2.jpg,jpg,720,720,1.000000,3,Airbus,A320
3,Aegean_Airlines_Airbus_A320-200_3.jpg,jpg,720,960,0.750000,3,Airbus,A320
4,Aegean_Airlines_Airbus_A320-200_4.jpg,jpg,720,540,1.333333,3,Airbus,A320


In [24]:
# Save as csv
path_tr =  './../CSV_annotate/SEATGURU/'
os.makedirs(path_tr, exist_ok=True)
imgs_df.to_csv(path_or_buf=path_tr + 'g7_SEATGURU.csv', sep=';', encoding='utf-8', index=False)

## Descriptive statistics

In [7]:
print(f'{len(np.unique(imgs_df.format))} unique image format(s).')
print(f'{len(np.unique(imgs_df.ncol))} unique ncol(s).')
print(f'{len(np.unique(imgs_df.height_to_width))} unique height_to_width(s).')
print(f'{len(np.unique(imgs_df.aircraft_type))} unique aircraft type(s).')

1 unique image format(s).
1 unique ncol(s).
128 unique height_to_width(s).
20 unique aircraft type(s).


In [8]:
print(f'{len(np.arange(nb_images)[imgs_df.aircraft_manufacturer == "Airbus"])} Airbus labelled images.')
print(f'{len(np.arange(nb_images)[imgs_df.aircraft_manufacturer == "Boeing"])} Boeing labelled images.')
print(f'{len(np.arange(nb_images)[imgs_df.aircraft_manufacturer == "Other"])} others.')

1043 Airbus labelled images.
1112 Boeing labelled images.
401 others.


In [9]:
def get_stats(imgs_df: pd.DataFrame, col: pd.Series, col_name: str):

    print(f'Max {col_name}: {np.max(col)}')
    print(f'Median {col_name}: {np.median(col)}')
    print(f'Min {col_name}: {np.min(col)}')
    print('')

In [10]:
# Dimensions
get_stats(imgs_df=imgs_df, col=imgs_df.height_to_width,
          col_name='height_to_width')

get_stats(imgs_df=imgs_df, col=imgs_df.height,
          col_name='height')

get_stats(imgs_df=imgs_df, col=imgs_df.width,
          col_name='width')

Max height_to_width: 3.898823529411765
Median height_to_width: 1.0
Min height_to_width: 0.2760416666666667

Max height: 1657
Median height: 720.0
Min height: 265

Max width: 960
Median width: 720.0
Min width: 337



In [11]:
# Number of images per aircraft type
pd.DataFrame(
    pd.pivot_table(imgs_df,
    index=['aircraft_manufacturer', 'aircraft_type'],
    aggfunc='count').format.sort_values(ascending=False))

,,format
aircraft_manufacturer,aircraft_type,
Other,,401
Boeing,777,344
Airbus,A330,331
Boeing,737,298
Airbus,A320,244
Boeing,787,198
Airbus,A321,152
Boeing,767,131
Airbus,A319,110
